In [70]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score,make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVR

In [71]:
df=pd.read_csv('processed.csv')
print('Data shape:',df.shape)

Data shape: (467, 7)


In [72]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values
print(X.shape)
print(y.shape)

(467, 6)
(467,)


In [73]:
C_values=list(np.logspace(-3,3,7)) + [200]
gamma_values=list(np.logspace(-3,3,7)) + ['scale','auto']
param_grid=[
    {
        'kernel': ['rbf','sigmoid','linear'],
        'C': C_values,
        'gamma': gamma_values,
        'epsilon': [1e-2,.1,1],
        'shrinking': [True,False]
    },
    {
        'kernel': ['poly'],
        'C': C_values,
        # 'gamma': gamma_values,
        'degree': [3,4,5],
        'epsilon': [1e-2,.1,1],
        'shrinking': [True,False]
    }
]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

def train_model(X_train,y_train):
    reg = SVR()
    grid=GridSearchCV(reg,param_grid,cv=5,scoring=make_scorer(r2_score),n_jobs=2)
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid

In [74]:
model=train_model(X_train,y_train)

{'C': 200, 'epsilon': 1, 'gamma': 'auto', 'kernel': 'rbf', 'shrinking': True}
0.8460150042405947


In [75]:
best_model=SVR(**model.best_params_)
best_model.fit(X_train,y_train)
print(r2_score(y_test,best_model.predict(X_test)))

0.8401462544398429
